In [1]:
import os  # Importar o módulo os
import serial
import serial.tools.list_ports
import time
import csv
import pandas as pd
import numpy as np
import joblib  # Para carregar o modelo
from keras.models import load_model
import paho.mqtt.client as mqtt
import json
from datetime import datetime
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


In [2]:
# Constantes
HANDSHAKE = 0
DATA_REQUEST = 1
N = 1  # Número de pontos coletados por vez
DELAY = 1000  # Delay de coleta de dados em milissegundos
TEMPERATURE_LIMIT = 80  # Exemplo de limite de temperatura
PWM_DECREMENT = 10  # Valor para decrementar o PWM


In [3]:
# Configurações de conexão MQTT para o TagoIO
mqtt_broker = "mqtt.tago.io"
mqtt_port = 1883
mqtt_username = "Token"
mqtt_password = "047aef83-5793-4f06-a21e-ca702df33116"
mqtt_topic = "DigitalTwin"


In [4]:
# Função para realizar o handshake com o Arduino
def handshake_arduino(arduino, sleep_time=1, print_handshake_message=False, handshake_code=0):
    arduino.close()
    arduino.open()
    time.sleep(sleep_time)
    timeout = arduino.timeout
    arduino.timeout = 2
    _ = arduino.read_all()
    arduino.write(bytes([handshake_code]))
    handshake_message = arduino.read_until()
    arduino.write(bytes([handshake_code]))
    handshake_message = arduino.read_until()
    if print_handshake_message:
        print("Handshake message: " + handshake_message.decode())
    arduino.timeout = timeout


In [5]:
#Converter byte em número
def parse_raw(raw):
    """Parse bytes output from Arduino."""
    raw = raw.decode()
    if raw[-1] != "\n":
        raise ValueError(
            "Input must end with newline, otherwise message is incomplete."
        )

    tempo, pwm, amb_t, mos_t = raw.rstrip().split(",")

    return int(tempo), int(pwm), float(amb_t), float(mos_t)

In [6]:
#Solicita a leitura do Arduino
def request_single_data(arduino):
    """Solicitar ao Arduino um único ponto de dados"""
    try:
        # Solicita dados ao arduino
        arduino.write(bytes([DATA_REQUEST]))

        # Ler os dados
        raw = arduino.read_until()
        print(raw)
        return parse_raw(raw)
    except Exception as e:
        print('Erro na leitura:', e)
        return None


In [7]:
# Configuração da porta serial
ports = serial.tools.list_ports.comports()
port = ports[0].device
arduino = serial.Serial(port, baudrate=115200, timeout=1)


In [8]:
# Realizar handshake com o Arduino
handshake_arduino(arduino, print_handshake_message=True, handshake_code=HANDSHAKE)


Handshake message: Energia parasita está: DESLIGADA



In [9]:
# Função para armazenar dados em CSV
def store_data(data, filename='teste.csv'):
    with open(filename, mode='a+', newline='') as file:
        write = csv.writer(file)
        write.writerows(data)

In [10]:
for _ in range(6):
    request_single_data(arduino)

b'Endere\xc3\xa7o do Dispositivo 0: 28616408E90E8A9F\r\n'
Erro na leitura: not enough values to unpack (expected 4, got 1)
b'Endere\xc3\xa7o do Dispositivo 1: 28616408E901BF06\r\n'
Erro na leitura: not enough values to unpack (expected 4, got 1)
b'Resolu\xc3\xa7\xc3\xa3o do Dispositivo 0: 12\r\n'
Erro na leitura: not enough values to unpack (expected 4, got 1)
b'Resolu\xc3\xa7\xc3\xa3o do Dispositivo 1: 12\r\n'
Erro na leitura: not enough values to unpack (expected 4, got 1)
b'Mensagem de handshake recebida.\r\n'
Erro na leitura: not enough values to unpack (expected 4, got 1)
b'3124,0,25.25,24.00\r\n'


In [11]:
model_path = 'lstm_model.keras'
# Carregar o modelo LSTM
lstm_model = load_model(model_path)


C:\Users\ebrwer\AppData\Local\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 7 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [13]:
# Coleta inicial de 10 dados para garantir dados suficientes para predição
initial_data = []
while len(initial_data) < 10:
    linha = request_single_data(arduino)
    if linha:
        initial_data.append(linha)
    time.sleep(DELAY / 1000)

# Armazenar os dados iniciais no CSV
store_data(initial_data)


b'13411,0,25.25,24.06\r\n'
b'14412,0,25.31,24.06\r\n'
b'15414,0,25.25,24.00\r\n'
b'16414,0,25.31,24.00\r\n'
b'17416,0,25.25,24.06\r\n'
b'35476,0,25.38,24.13\r\n'
b'36477,0,25.38,24.19\r\n'
b'37479,0,25.38,24.19\r\n'
b'38479,0,25.38,24.19\r\n'
b'39480,0,25.38,24.19\r\n'


In [14]:

        # Evento chamado quando a conexão MQTT é estabelecida
        def on_connect(client, userdata, flags, rc):
            print("Conectado ao broker MQTT com código de resultado: " + str(rc))
        
        # Inicializa o cliente MQTT
        client = mqtt.Client(client_id="csv_publisher")

        # Define o callback de conexão
        client.on_connect = on_connect

        # Configure o usuário e senha do MQTT
        client.username_pw_set(mqtt_username, mqtt_password)

        # Conecta-se ao broker MQTT
        client.connect(mqtt_broker, mqtt_port, 60)


0

In [15]:
# Loop principal para coleta contínua de dados e previsão
while True:
    data = []
    for _ in range(N):
        linha = request_single_data(arduino)
        if linha:
            data.append(linha)
        time.sleep(DELAY / 1000)

    # Armazenar dados em CSV
    # store_data(data)

    # Carregar dados do CSV
    dataT = pd.read_csv('teste.csv', sep=',', decimal='.')

    # Verificar se há dados suficientes para a previsão
    if len(dataT) >= 10:  # Certificar-se de que há pelo menos `seq_length` linhas de dados
        # Prever os próximos 10 passos
        future_steps = 20
        seq_length = 11
        scaler = MinMaxScaler()
        scaled_data = scaler.fit_transform(dataT[['pwm', 'amb_t', 'mos_t']])
        last_sequence = scaled_data[-seq_length:]
        future_predictions = []
        for _ in range(future_steps):
            prediction = lstm_model.predict(last_sequence.reshape(1, seq_length, 3), verbose=0)
            future_predictions.append(prediction[0])
            last_sequence = np.vstack([last_sequence[1:], prediction])

        # Inverter a normalização das previsões
        future_predictions = scaler.inverse_transform(future_predictions)

        # Obter o último valor da última predição
        ultimo_valor = future_predictions[-1, -1]

        # Limitar as casas decimais para 2
        ultimo_valor_formatado = round(ultimo_valor, 2)

        # Calcular o erro entre o valor real de mos_t e a previsão
        mos_t_real = data[-1][3]
        erro = mos_t_real - ultimo_valor_formatado

        # Adicionar o valor previsto como a quinta coluna nos dados coletados
        data_com_previsao = [list(linha) + [ultimo_valor_formatado, erro] for linha in data]

        # Armazenar os dados com a previsão no CSV
        # store_data(data_com_previsao)

        print("Último valor da previsão futura:", ultimo_valor_formatado)
        print("Erro:", erro)

        # Preparar os dados para o envio ao TagoIO
        tempo = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        pwm = data[-1][1]  # Último valor de PWM
        amb_t = data[-1][2]  # Último valor de amb_t
        mos_t = data[-1][3]  # Último valor de mos_t

        payload_list = [
            {"variable": "pwm", "value": float(pwm), "time": tempo},
            {"variable": "amb_t", "value": amb_t, "time": tempo},
            {"variable": "mos_t", "value": mos_t, "time": tempo},
            {"variable": "prediction", "value": ultimo_valor_formatado, "time": tempo},
            {"variable": "error", "value": erro, "time": tempo}
        ]

        for payload in payload_list:
            # Publica o payload no tópico MQTT
            client.publish(mqtt_topic, json.dumps(payload))
            print(f"Dados publicados no tópico MQTT {mqtt_topic}: {payload}")

    else:
        print("Dados insuficientes para previsão.")


b'40481,0,25.38,24.19\r\n'
Último valor da previsão futura: 24.82
Erro: -0.629999999999999
Dados publicados no tópico MQTT DigitalTwin: {'variable': 'pwm', 'value': 0.0, 'time': '2024-06-11 19:53:09'}
Dados publicados no tópico MQTT DigitalTwin: {'variable': 'amb_t', 'value': 25.38, 'time': '2024-06-11 19:53:09'}
Dados publicados no tópico MQTT DigitalTwin: {'variable': 'mos_t', 'value': 24.19, 'time': '2024-06-11 19:53:09'}
Dados publicados no tópico MQTT DigitalTwin: {'variable': 'prediction', 'value': 24.82, 'time': '2024-06-11 19:53:09'}
Dados publicados no tópico MQTT DigitalTwin: {'variable': 'error', 'value': -0.629999999999999, 'time': '2024-06-11 19:53:09'}
b'41482,0,25.38,24.19\r\n'
Último valor da previsão futura: 24.82
Erro: -0.629999999999999
Dados publicados no tópico MQTT DigitalTwin: {'variable': 'pwm', 'value': 0.0, 'time': '2024-06-11 19:53:12'}
Dados publicados no tópico MQTT DigitalTwin: {'variable': 'amb_t', 'value': 25.38, 'time': '2024-06-11 19:53:12'}
Dados publ

KeyboardInterrupt: 

In [14]:
# Loop principal para coleta contínua de dados e previsão
while True:
    data = []
    for _ in range(N):
        linha = request_single_data(arduino)
        if linha:
            data.append(linha)
        time.sleep(DELAY / 1000)

    # Armazenar dados em CSV
    #store_data(data)

    # Carregar dados do CSV
    dataT = pd.read_csv('teste.csv', sep=',', decimal='.')

    # Verificar se há dados suficientes para a previsão
    if len(dataT) >= 10:  # Certificar-se de que há pelo menos `seq_length` linhas de dados
        # Prever os próximos 10 passos
        future_steps = 20
        seq_length = 11
        scaler = MinMaxScaler()
        scaled_data = scaler.fit_transform(dataT[['pwm', 'amb_t', 'mos_t']])
        last_sequence = scaled_data[-seq_length:]
        future_predictions = []
        for _ in range(future_steps):
            prediction = lstm_model.predict(last_sequence.reshape(1, seq_length, 3),verbose=0)
            future_predictions.append(prediction[0])
            last_sequence = np.vstack([last_sequence[1:], prediction])

        # Inverter a normalização das previsões
        future_predictions = scaler.inverse_transform(future_predictions)

        # Obter o último valor da última predição
        ultimo_valor = future_predictions[-1, -1]

        # Limitar as casas decimais para 2
        ultimo_valor_formatado = round(ultimo_valor, 2)

        # Adicionar o valor previsto como a quinta coluna nos dados coletados
        data_com_previsao = [list(linha) + [ultimo_valor_formatado] for linha in data]

        # Armazenar os dados com a previsão no CSV
        store_data(data_com_previsao)
        
        print("Último valor da previsão futura:", ultimo_valor_formatado)

            # Preparar os dados para o envio ao TagoIO
        tempo = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        pwm = data[-1][1]  # Último valor de PWM
        amb_t = data[-1][2]  # Último valor de amb_t
        mos_t = data[-1][3]  # Último valor de mos_t

        payload_list = [
            {"variable": "pwm", "value": float(pwm), "time": tempo},
            {"variable": "amb_t", "value": amb_t, "time": tempo},
            {"variable": "mos_t", "value": mos_t, "time": tempo},
            {"variable": "prediction", "value": ultimo_valor_formatado, "time": tempo}
        ]

        for payload in payload_list:
            # Publica o payload no tópico MQTT
            client.publish(mqtt_topic, json.dumps(payload))
            print(f"Dados publicados no tópico MQTT {mqtt_topic}: {payload}")

    else:
        print("Dados insuficientes para previsão.")

b'5497,0,28.50,64.31\r\n'
Último valor da previsão futura: 62.82
Dados publicados no tópico MQTT DigitalTwin: {'variable': 'pwm', 'value': 0.0, 'time': '2024-06-11 18:07:10'}
Dados publicados no tópico MQTT DigitalTwin: {'variable': 'amb_t', 'value': 28.5, 'time': '2024-06-11 18:07:10'}
Dados publicados no tópico MQTT DigitalTwin: {'variable': 'mos_t', 'value': 64.31, 'time': '2024-06-11 18:07:10'}
Dados publicados no tópico MQTT DigitalTwin: {'variable': 'prediction', 'value': 62.82, 'time': '2024-06-11 18:07:10'}
b'6498,0,28.50,63.81\r\n'
Último valor da previsão futura: 62.09
Dados publicados no tópico MQTT DigitalTwin: {'variable': 'pwm', 'value': 0.0, 'time': '2024-06-11 18:07:11'}
Dados publicados no tópico MQTT DigitalTwin: {'variable': 'amb_t', 'value': 28.5, 'time': '2024-06-11 18:07:11'}
Dados publicados no tópico MQTT DigitalTwin: {'variable': 'mos_t', 'value': 63.81, 'time': '2024-06-11 18:07:11'}
Dados publicados no tópico MQTT DigitalTwin: {'variable': 'prediction', 'valu

KeyboardInterrupt: 

In [15]:
# Desconecta-se do broker MQTT e do arduino
client.disconnect()
arduino.close()